# Project 1 - IR CIn UFPE 2022.1

Author:

Matheus Rodrigues de Souza Félix (matheusrdgsf@gmail.com) \
Rodrigo Melo

### Libs

In [4]:
import pandas as pd

### Data Collect

In [12]:
data = pd.read_json(path_or_buf="./scidocs-generated-queries/train.jsonl", lines=True)

In [13]:
data

,_id,title,text,query
0,632589828c8b9fca2c3a59e97451fde8fa7d188d,A hybrid of genetic algorithm and particle swa...,An evolutionary recurrent network which automa...,what is gappso?
1,632589828c8b9fca2c3a59e97451fde8fa7d188d,A hybrid of genetic algorithm and particle swa...,An evolutionary recurrent network which automa...,what is recurrent neural networks
2,632589828c8b9fca2c3a59e97451fde8fa7d188d,A hybrid of genetic algorithm and particle swa...,An evolutionary recurrent network which automa...,what is hgapso
3,86e87db2dab958f1bd5877dc7d5b8105d6e31e46,A Hybrid EP and SQP for Dynamic Economic Dispa...,Dynamic economic dispatch (DED) is one of the ...,what is the ep used for
4,86e87db2dab958f1bd5877dc7d5b8105d6e31e46,A Hybrid EP and SQP for Dynamic Economic Dispa...,Dynamic economic dispatch (DED) is one of the ...,what is dynamic economic dispatch
...,...,...,...,...
74788,2a4423b10725e54ad72f4f1fcf77db5bc835f0a6,Optimization by simulated annealing.,There is a deep and useful connection between ...,what is the theory of annealing
74789,2a4423b10725e54ad72f4f1fcf77db5bc835f0a6,Optimization by simulated annealing.,There is a deep and useful connection between ...,annealing in solids definition
74790,dec997b20ebe2b867f68cc5c123d9cb9eafad6bb,Deriving optimal weights in deep neural networks,Training deep neural networks generally requir...,how to train neural networks
74791,dec997b20ebe2b867f68cc5c123d9cb9eafad6bb,Deriving optimal weights in deep neural networks,Training deep neural networks generally requir...,can you derive weights from gradient descent


## Pre-rocesing Data

In this step we will create 5 data models:

v1 - Only Tokenization; \
v2 - Only Stopword Filter; \
v3 - Only Stemming; \
v4 - Remove Stopwords and Stemming; \
v5 - Remove Stopworpd and expand words with Synonyms.


fatal: cannot mkdir https://github.com/MatheusRDG/information-retrieval-cin-2022: Invalid argument


### Vectorizing Data

### Evaluation